# [RecipeBox](https://eightportions.com/datasets/Recipes/)

Analysis of dataset should contain: 
* [Plantuml](https://plantuml.com/de/json) diagram of json structure 
* Amount of recipes
* Plots:
 * Num of Instructions
 * Num of Ingredients
 * Most used words in Instructions
 * Most used words in Ingedients
 * (Optional) Kind of dish


In [1]:
import json
import re

In [2]:
file_ar = open('.recipeBox/recipes_raw_nosource_ar.json')
data_ar = json.load(file_ar)

file_epi = open('.recipeBox/recipes_raw_nosource_epi.json')
data_epi = json.load(file_epi)

file_fn = open('.recipeBox/recipes_raw_nosource_fn.json')
data_fn = json.load(file_fn)

data = {**data_ar, **data_epi, **data_fn}

In [4]:
data_stripped = {k: v for k, v in data.items() if v}

In [8]:
# Analysis
## Length of the dataset
print('Amount of recipes: ' + str(len(data_stripped)))


sumInstructions = 0
sumIngredients = 0
for value in data_stripped.values():
    sumIngredients = sumIngredients + len(value['ingredients'])
    if value['instructions']:
        sumInstructions = sumInstructions + len(value['instructions'].split("."))

print('Amount of Ingredients: ' + str(sumIngredients))
print('Amount of Instructions: ' + str(sumInstructions))

Amount of recipes: 124647
Amount of Ingredients: 1316950
Amount of Instructions: 1924048


In [20]:
ingredientsDict = {}

for value in data_stripped.values():
    for ingredient in value['ingredients']:
        charsToRemove = ",.*®©™()[]"
        ingredientStripped = re.sub("[" + charsToRemove + "]", "", ingredient.lower())
        for word in ingredientStripped.split():
            ingredientsDict[word] = ingredientsDict.setdefault(word, 0) + 1

print(len(ingredientsDict))
sortedIngredientsDict = sorted(ingredientsDict.items(), key=lambda x:x[1])
print(sortedIngredientsDict)

24947
[('rest:', 1), ('together:', 1), ('4-x6-inch', 1), ('swan', 1), ('grinders:', 1), ('three-berry', 1), ('roast-ready', 1), ('activedry', 1), ("kingsford's", 1), ('assemble:', 1), ('mexi-ranch', 1), ('butter-margarine', 1), ('goodies', 1), ('biga:', 1), ('two-cheese', 1), ('dramatically;', 1), ('centers:', 1), ('f/32', 1), ('heels', 1), ('sausagemeat', 1), ('squash:', 1), ('lup', 1), ('bavarian-style', 1), ('frangipane:', 1), ('pesto-alfredo', 1), ('freshly-cracked', 1), ('combinbation', 1), ('garlic-and-tomato', 1), ('mushrooms-stems', 1), ('solution:', 1), ('high-heat-resistant', 1), ("petes's", 1), ('trouble', 1), ('crimini;', 1), ('robusto', 1), ('glass:', 1), ('blood:', 1), ('super-hot', 1), ('shells:', 1), ('muffin:', 1), ("lars'", 1), ('syrup-packed', 1), ('lemonade-flavor', 1), ('poolish:', 1), ('pearlini', 1), ('jar:', 1), ('chantaboon', 1), ('williamsburg', 1), ('gypsy', 1), ('part-skim-milk', 1), ('garlic-and-butter', 1), ('gf', 1), ('coffeecake', 1), ('jam;', 1), ('pres